# OCR using Kraken

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import patches

from PIL import Image
from kraken.binarization import nlbin
from kraken.kraken import 

In [ ]:
plt.rcParams["figure.figsize"] = (20,15)

### Open Image

In [ ]:
img = Image.open("../demo_data/demo_image.jpeg")

plt.imshow(img)

## OCR using CLI

In [ ]:
# Get required model for the lab
!kraken get 10.5281/zenodo.7631619 

In [ ]:
!kraken -i "../demo_data/demo_image.jpeg" output.txt segment -bl ocr -m cremma-generic-1.0.1.mlmodel

## Compare to ground truth

In [ ]:
# Load ground truth
with open("../demo_data/ground_truth.txt", encoding="utf-8") as f:
    ground_truth = f.readlines()

# Load kraken prediction
with open("output.txt", encoding="utf-8") as f:
    output = f.readlines()

In [ ]:
ground_truth

In [ ]:
output

**Exercice**: 
1. Print alternatively the ground truth and the output of Kraken and visually compare the results.
2. Compute the ratio of succesfully recognized words divided by total number of words. A word is successfully predicted when all the letters are guessed and in the right order.
3. What does this formula reminds you of ?

## Training our own model

Basic syntax for training using ketos and specifying an architecture is:

`ketos train -f alto -s '[1,0,0,3 Cr3,3,16 Mp3,3 Lfys64 Lbx128 Lbx256 Do]' ../training_data/*.xml`

You can test the model using the `ketos test` command:

`ketos test -m '/Users/mimsy/Library/Application Support/kraken/cremma-generic-1.0.1.mlmodel' -f alto ../demo_data/*.xml`.

(where the model is located depends on your OS and the output of the function `get_app_dir` of click).

**Exercice**: 
1. Train the neural network using given architecture.
2. Provide accuracy of the model.
3. Evalute the performance of the model we used at the beginning of the session on the Alto XML example data located in `demo_data` and of the model you just trained.

In [ ]:
!ketos train -f alto -s '[1,0,0,3 Cr3,3,16 Lfys64 Do]' ../training_data/*.xml

## Generating Alto XML format
For large tasks, UI is easier to manipulate, but Kraken can be sufficient for small edits.

You need to add the `-a` option at the beginning of the command. The output.txt will then be an ALTO XML file.

In [ ]:
!kraken -a -i "../demo_data/demo_image.jpeg" output.txt segment -bl ocr -m cremma-generic-1.0.1.mlmodel 